In [5]:
import pandas as pd
import matplotlib.pyplot as mp

In [2]:
arquivo_csv = r'C:\Users\kalve\Documents\damage-controller\backend\Banco_de_Dados 2.CSV'
df = pd.read_csv(arquivo_csv, encoding='latin1', sep=';', on_bad_lines='skip')

C:\Users\kalve\AppData\Local\Temp\ipykernel_28244\140171325.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo_csv, encoding='latin1', sep=';', on_bad_lines='skip')


In [4]:
contador_def=df['DEFEITOS'].value_counts()
print(contador_def)

DEFEITOS
NÃO AJUSTA            2263
DESCASCADO            1745
DANIFICADO            1235
O SEM DEFEITO         1174
RISCO                  984
                      ... 
VINCO - ESTRIA           1
CASCA DE LARANJA         1
DEFEITO DE INJEÇÃO       1
ONDULAÇÃO                1
MÁ REGULAGEM             1
Name: count, Length: 67, dtype: int64
